Pratibha Dixith (pdixit5)




In [8]:
import csv
import re
import nltk
from nltk.stem import PorterStemmer
from pyspark import SparkContext
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.evaluation  import BinaryClassificationMetrics
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

#from nltk.corpus import stopwords

In [9]:
#csv.reader(open("train.csv",'r',encoding='ISO-8859-1'))
#sc = SparkContext()

stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']
tweetsentilist = []
featurelist = []
wordlist = []

In [10]:


def cleantweets(eachtweet):
    stemmer=PorterStemmer() 
    #for eachtweet in onlytweets:
    #print(eachtweet)
    if 'http' in eachtweet:
        #print(eachtweet)
        eachtweet=re.sub('https?://[^\s]+','URL',eachtweet)
    if 'www.' in eachtweet:
        eachtweet=re.sub('www\.[^\s]+','URL',eachtweet)
    if '@' in eachtweet:
        eachtweet=re.sub('@[^\s]+','AT_USER',eachtweet)
    #eachtweet=re.findall(r"[\w']+", eachtweet)
    tweetwordlist=eachtweet.strip().split(' ')
    eachtweet=''
    for eachword in tweetwordlist:
        #print(eachword)
        #print(len(eachword))
        eachword=re.sub('[^\w]+','',eachword)
        pattern = re.compile(r'(.)\1{1,}', re.DOTALL) 
        eachword =  pattern.sub(r'\1\1', eachword)
        #print(eachword)
        eachword=stemmer.stem(eachword) 
        #print(eachword)
            
        if eachword.startswith('#'):
            eachword=eachword.lstrip('#')
        if  len(eachword)!=0 :
            if not eachword[0].isalpha():
                #print(eachword)
                eachword=''
        if eachword not in stopwords:
            if eachtweet=='':
                eachtweet=eachword
            elif eachword=='':
                eachtweet=eachtweet
            else:
                eachtweet=eachtweet+' '+eachword
    return eachtweet

In [11]:
def perdictiontruepositive(predictiontweet):
    if predictiontweet[0] == 1 and predictiontweet[1]==1:
        return predictiontweet
    
def perdictiontruenegative(predictiontweet):
    if predictiontweet[0] == 0 and predictiontweet[1]==0:
        return predictiontweet

def perdictionfalsepositive(predictiontweet):
    if predictiontweet[0] == 1 and predictiontweet[1]==0:
        return predictiontweet
    
def perdictionfalsenegative(predictiontweet):
    if predictiontweet[0] == 0 and predictiontweet[1]==1:
        return predictiontweet

In [12]:
def metrics(predictiontweet):

    predicttruepositive = predictiontweet.filter(perdictiontruepositive)
    predictflasepositive = predictiontweet.filter(perdictionfalsepositive)
    print("predicttruepositive:" + str(predicttruepositive.count()))
    print("predictflasepositive:"+str(predictflasepositive.count()))
    precision = predicttruepositive.count()/(predicttruepositive.count()+predictflasepositive.count())
    print("precision : "+str(precision))

    predictflasenegative = predictiontweet.filter(perdictionfalsenegative)
    predicttruenegative = predictiontweet.filter(perdictiontruenegative)
    print("predictflasenegative:" + str(predictflasenegative.count()))
    print("predicttruenegative:"+str(predicttruenegative.count()))
    recall = predicttruepositive.count()/(predicttruepositive.count()+predictflasenegative.count())

    print("recall : "+str(recall))

    fmeasure = 2*(precision*recall)/(recall+precision)
    
    perfmetrics = BinaryClassificationMetrics(predictiontweet)

    print("fmeasure: "+str(fmeasure))
    print("area under ROC: "+str(perfmetrics.areaUnderROC))


In [ ]:

def plot(naivedata,lrdata,dtdata):

    #y = [81.3175, 87.36, 62.2187]
    #x = [74.72, 71.825, 62.12]
    #z = [79.38, 77.99, 60.16]
    N = len(naivedata)
#x = range(N)
    ind = np.arange(N)
    plt.figure(figsize=(10,10))
    plt.ylim(0,100)
    ax = plt.subplot(111)
    #width = 1/1.5
    rects1 = ax.bar(ind-0.2, naivedata, width=0.2, color="blue")
    rects2 = ax.bar(ind, lrdata, width=0.2, color="red")
    rects3 = ax.bar(ind+.2, dtdata, width=0.2, color="green")
    ax.set_ylabel('Accuracy')
    ax.set_title('Accuracy comparison between all classification')
    ax.set_xticks(ind)
    ax.set_xticklabels(('Training Accuracy', 'K-fold mean Accuracy', 'Test Accuracy'))
    ax.legend((rects1[0], rects2[0],rects3[0]), ('Naive-Baiye', 'Logistic Regression','Decision Tree'))


#fig = plt.gcf()
#plot_url = py.plot_mpl(fig, filename='mpl-basic-bar')

In [ ]:
def featurevector(row):
    row=row.split('\",')
    tweet = cleantweets(row[5].lower())
    return (row[0].lstrip('"'),tweet.split())

def tweettext(row):
    line=row.split('\",')
    return (line[5])
            

tweetsenti=sc.textFile('train.csv').map(featurevector)    
tweetsenti.persist()
tweetobj = sc.textFile('test.csv').map(tweettext)
testsenti=sc.textFile('test.csv').map(featurevector)   
testsenti.persist()
#print(tweetsentilist)
#for x in tweetsenti.collect():
 #   print(x)
    
naiveplot = []
lrplot = []
dtplot = []
    
htf = HashingTF(50000)

tweettrainhash=tweetsenti.map(lambda senti:LabeledPoint(float(senti[0]),htf.transform(senti[1])))
testtrainhash = testsenti.map(lambda senti:LabeledPoint(float(senti[0]),htf.transform(senti[1])))
train_hashed=tweettrainhash.randomSplit([0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])
#train_hashed,test_hashed=tweettrainhash.randomSplit([0.7,0.3])


naiveBayesmodel = NaiveBayes.train(tweettrainhash)
predictiontweet = tweettrainhash.map(lambda pointvalue: (float(naiveBayesmodel.predict(pointvalue.features)),pointvalue.label))
accuracy = predictiontweet.filter(lambda prediction:prediction[0]==prediction[1]).count()/float(tweettrainhash.count())
naiveplot.append(accuracy*100)
print("Naive-Bayes classifier train-train accuracy: "+str(accuracy*100) )


mean=0.0
for i in range(0,10):
    traindata= sc.emptyRDD()
    testdata=train_hashed[i]
    for cnt in range(0,10):
        if cnt!=i:
            traindata=traindata+train_hashed[cnt]
    naiveBayesmodel = NaiveBayes.train(traindata)
    #logisticregressionmodel = LogisticRegressionWithLBFGS.train(traindata)
    #decisiontreenmodel = DecisionTree.trainClassifier(traindata, numClasses=2, categoricalFeaturesInfo={},impurity='gini', maxDepth=5, maxBins=32)
    #predictions = decisiontreenmodel.predict(testdata.map(lambda x: x.features))
    #predictiontweet = testdata.map(lambda lp:lp.label).zip(predictions)
    predictiontweet = testdata.map(lambda pointvalue: (float(naiveBayesmodel.predict(pointvalue.features)),pointvalue.label))
    #print(predictiontweet.take(100))
    accuracy = predictiontweet.filter(lambda prediction:prediction[0]==prediction[1]).count()/float(testdata.count())
    print("Naive-Bayes classifier K-fold accuracy: "+str(accuracy*100) )
    #cnt=cnt+1
    mean=mean+accuracy
naiveplot.append((mean/10)*100)
print("Naive-Bayes classifier average for K-fold classifier: "+str((mean/10)*100) )  

naiveBayesmodel = NaiveBayes.train(tweettrainhash)
predictiontweet = testtrainhash.map(lambda pointvalue: (float(naiveBayesmodel.predict(pointvalue.features)),pointvalue.label))
accuracy = predictiontweet.filter(lambda prediction:prediction[0]==prediction[1]).count()/float(testtrainhash.count())
naiveplot.append(accuracy*100)
print("Naive-Bayes classifier train-test accuracy: "+str(accuracy*100) )

print("Performance metrics for Naive-Bayes")
metrics(predictiontweet)

logisticregressionmodel = LogisticRegressionWithLBFGS.train(tweettrainhash)
predictiontweet = tweettrainhash.map(lambda pointvalue: (float(logisticregressionmodel.predict(pointvalue.features)),pointvalue.label))
accuracy = predictiontweet.filter(lambda prediction:prediction[0]==prediction[1]).count()/float(tweettrainhash.count())
lrplot.append(accuracy*100)
print("Logistic Regression train-train classifier: "+str(accuracy*100) )

mean=0.0
for i in range(0,10):
    traindata= sc.emptyRDD()
    testdata=train_hashed[i]
    for cnt in range(0,10):
        if cnt!=i:
            traindata=traindata+train_hashed[cnt]
    logisticregressionmodel = LogisticRegressionWithLBFGS.train(traindata)
    predictiontweet = testdata.map(lambda pointvalue: (float(logisticregressionmodel.predict(pointvalue.features)),pointvalue.label))
    accuracy = predictiontweet.filter(lambda prediction:prediction[0]==prediction[1]).count()/float(testdata.count())
    print("Logistic Regression classifier K-fold accuracy: "+str(accuracy*100) )
    mean=mean+accuracy
lrplot.append((mean/10)*100)
print("Logistic Regression classifier average for K-fold classifier: "+str((mean/10)*100) )    


logisticregressionmodel = LogisticRegressionWithLBFGS.train(tweettrainhash)
predictiontweet = testtrainhash.map(lambda pointvalue: (float(logisticregressionmodel.predict(pointvalue.features)),pointvalue.label))
accuracy = predictiontweet.filter(lambda prediction:prediction[0]==prediction[1]).count()/float(testtrainhash.count())
lrplot.append(accuracy*100)
print("Logistic Regression train-test classifier: "+str(accuracy*100) )

print("Performance metrics for Logistic Regression")
metrics(predictiontweet)

#train_hashed=tweettrainhash.randomSplit([0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])
#mean=0.0
  
    
#print(tweetsentilist)        
#print (onlytweets)

htfdectree = HashingTF(5000)
dectweettrainhash=tweetsenti.map(lambda senti:LabeledPoint(senti[0],htfdectree.transform(senti[1])))
dectesttrainhash = testsenti.map(lambda senti:LabeledPoint(senti[0],htfdectree.transform(senti[1])))
train_hashed=dectweettrainhash.randomSplit([0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])


decisiontreenmodel = DecisionTree.trainClassifier(dectweettrainhash, numClasses=2, categoricalFeaturesInfo={},impurity='gini', maxDepth=5, maxBins=32)
predictions = decisiontreenmodel.predict(dectweettrainhash.map(lambda x: x.features))
predictiontweet = dectweettrainhash.map(lambda lp:lp.label).zip(predictions)
accuracy = predictiontweet.filter(lambda prediction:prediction[0]==prediction[1]).count()/float(dectweettrainhash.count())
dtplot.append(accuracy*100)
print("Decision tree train-train classifier: "+str(accuracy*100) )

mean=0.0
for i in range(0,10):
    traindata= sc.emptyRDD()
    testdata=train_hashed[i]
    for cnt in range(0,10):
        if cnt!=i:
            traindata=traindata+train_hashed[cnt]
    decisiontreenmodel = DecisionTree.trainClassifier(traindata, numClasses=2, categoricalFeaturesInfo={},impurity='gini', maxDepth=5, maxBins=32)
    predictions = decisiontreenmodel.predict(testdata.map(lambda x: x.features))
    predictiontweet = testdata.map(lambda lp:lp.label).zip(predictions)
    accuracy = predictiontweet.filter(lambda prediction:prediction[0]==prediction[1]).count()/float(testdata.count())
    print("Decision tree classifier K-fold accuracy: "+str(accuracy*100) )
    mean=mean+accuracy
dtplot.append(mean*100)
print("Decision tree Regression classifier anerage for K-fold classifier: "+str((mean/10)*100))


#dtplot.append(62)
#dtplot.append(62)
decisiontreenmodel = DecisionTree.trainClassifier(dectweettrainhash, numClasses=2, categoricalFeaturesInfo={},impurity='gini', maxDepth=5, maxBins=32)
predictions = decisiontreenmodel.predict(dectesttrainhash.map(lambda x: x.features))
predictiontweet = dectesttrainhash.map(lambda lp:lp.label).zip(predictions)
accuracy = predictiontweet.filter(lambda prediction:prediction[0]==prediction[1]).count()/float(dectesttrainhash.count())
dtplot.append(accuracy*100)
print("Decision tree train-test classifier: "+str(accuracy*100) )

print("Performance metrics for Decision tree")
metrics(predictiontweet)

##train_hashed,test_hashed=tweettrainhash.randomSplit([0.7,0.3])

plot(naiveplot,lrplot,dtplot)

logisticregressionmodel.clearThreshold()
predictiontweet = testtrainhash.map(lambda pointvalue: (float(logisticregressionmodel.predict(pointvalue.features)),pointvalue.label))
result = predictiontweet.zip(tweetobj)
correcttweet = result.filter(lambda x : (x[0][0] >=0.5 and x[0][1] == 1) or (x[0][0] < 0.5 and x[0][1] == 0) )
somecorrect = correcttweet.takeOrdered(20 , key = lambda p: p[0][0] )
print()
print("some 20 correctly classified tweet:")
for t in somecorrect:
    print(t[0][0],t[1])

print()
print("Top 5 correctly classified tweet:")
top5 = correcttweet.takeOrdered(5 , key = lambda p: -p[0][0] )
for t in top5:
    print(t[0][0],t[1])


correcttweet = result.filter(lambda x : (x[0][0] >=0.5 and x[0][1] == 0) or (x[0][0] < 0.5 and x[0][1] == 1) )
somecorrect = correcttweet.takeOrdered(20 , key = lambda p: p[0][0] )
print()
print("Some 20 incorrectly classified tweet:")
for t in somecorrect:
    print(t[0][0],t[1])
    
print()
print("Top 5 incorrectly classified tweet:")
top5 = correcttweet.takeOrdered(5 , key = lambda p: -p[0][0] )
for t in top5:
    print(t[0][0],t[1])





Naive-Bayes classifier train-train accuracy: 81.3175


In [ ]:

def perdictiontruepositiveprobrate(predictiontweet,thres):
    if predictiontweet[0] >= thres and predictiontweet[1]==1:
        return predictiontweet
    
def perdictiontruenegativeprobrate(predictiontweet,thres):
    if predictiontweet[0] < thres and predictiontweet[1]==0:
        return predictiontweet

def perdictionfalsepositiveprobrate(predictiontweet,thres):
    if predictiontweet[0] >= thres and predictiontweet[1]==0:
        return predictiontweet
    
def perdictionfalsenegativeprobrate(predictiontweet,thres):
    if predictiontweet[0] < thres and predictiontweet[1]==1:
        return predictiontweet

In [ ]:
def probmetrics(predictiontweet,thres):

    predicttruepositiveprob = predictiontweet.filter(lambda x:perdictiontruepositiveprobrate(x,thres))
    predictflasepositiveprob = predictiontweet.filter(lambda x:perdictionfalsepositiveprobrate(x,thres))
    predictflasenegative = predictiontweet.filter(lambda x:perdictionfalsenegativeprobrate(x,thres))
    predicttruenegative = predictiontweet.filter(lambda x:perdictiontruenegativeprobrate(x,thres))

    tpr = predicttruepositiveprob.count()/(predicttruepositiveprob.count()+predictflasenegative.count())
    fpr = predictflasepositiveprob.count()/(predictflasepositiveprob.count()+predicttruenegative.count())
    
    return (tpr,fpr)


In [ ]:
thresholdvalues = [0.0,0.25,0.5,0.75,1]
xlist = []
ylist = []

for i in range(len(thresholdvalues)):
    thres = thresholdvalues[i]
    listVal  = probmetrics(predictiontweet,thres)
    xlist.append(listVal[1])
    ylist.append(listVal[0])
plt.plot(xlist, ylist)
plt.plot([0.0,1],[0.0,1],'r--')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('Logistic Regression ROC')
plt.legend(('Logistic Regression','Line Of No Discrimination'),loc=4)